In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor,RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train_data=pd.read_csv('../Data/data-science-london-scikit-learn/train.csv', header=None)
test_data=pd.read_csv('../Data/data-science-london-scikit-learn/test.csv', header=None)
trainLabels=pd.read_csv('../Data/data-science-london-scikit-learn/trainLabels.csv', header=None)

In [5]:
print(train_data.shape)
print(test_data.shape)
print(trainLabels.shape)

(1000, 40)
(9000, 40)
(1000, 1)


In [6]:
trainLabels = np.ravel(trainLabels)
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(train_data, trainLabels, test_size=0.25, random_state=42)

In [7]:
clf = RandomForestClassifier(random_state=100)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("MSE = ",metrics.mean_squared_error(y_test, y_pred))

MSE =  0.124


In [8]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [10, 30, 50]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, n_jobs=-1,cv=10)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)


In [9]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
train_accuracy = grid_result.score(x_train, y_train)
print("RF: train accuracy={:.5f}".format(train_accuracy))

Best Accuracy: 0.861333 using {'max_depth': 10, 'n_estimators': 200}
RF: train accuracy=1.00000


In [10]:
# 使用最佳參數重新建立模型
clf_bestparam = RandomForestClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(test_data)

In [11]:
submission = {'Id': np.arange(1, y_pred.shape[0]+1),
             'Solution': y_pred}
submission = pd.DataFrame(submission)
submission

,Id,Solution
0,1,1
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
8995,8996,1
8996,8997,1
8997,8998,1
8998,8999,0


In [12]:
submission.to_csv('submission_rf.csv', index=False)